In [1]:
import copy
import datetime as dt
import importlib # needed so that we can reload packages
import matplotlib.pyplot as plt
import os, os.path
import numpy as np
import pandas as pd
import pathlib
import sys
import time
from typing import Union
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

##  IMPORT SISEPUEDE EXAMPLES AND TRANSFORMERS

from sisepuede.manager.sisepuede_examples import SISEPUEDEExamples
from sisepuede.manager.sisepuede_file_structure import SISEPUEDEFileStructure
import sisepuede.core.support_classes as sc
import sisepuede.transformers as trf
import sisepuede.utilities._plotting as spu
import sisepuede.utilities._toolbox as sf

file_struct = SISEPUEDEFileStructure()

matt = file_struct.model_attributes

In [2]:
# Define the current directory and output data path
dir_cur = pathlib.Path(os.getcwd()).parent
output_data = dir_cur.joinpath("ssp_run_output")

# Change this line only when you receive a new run
run_folder = "sisepuede_results_run_sisepuede_run_2025-07-23T16;44;34.512127"
file_name  = "sisepuede_results_sisepuede_run_2025-07-23T16;44;34.512127_WIDE_INPUTS_OUTPUTS.csv"

run = output_data.joinpath(run_folder)
tableau = dir_cur.joinpath("Tableau/data")

In [3]:
# Define the region
region = "tanzania"

In [4]:
# Load the output data
ssp_output = pd.read_csv(run.joinpath(file_name))
ssp_output.head()

,primary_id,region,time_period,area_agrc_crops_bevs_and_spices,area_agrc_crops_cereals,area_agrc_crops_fibers,area_agrc_crops_fruits,area_agrc_crops_herbs_and_other_perennial_crops,area_agrc_crops_nuts,area_agrc_crops_other_annual,...,yf_agrc_herbs_and_other_perennial_crops_tonne_ha,yf_agrc_nuts_tonne_ha,yf_agrc_other_annual_tonne_ha,yf_agrc_other_woody_perennial_tonne_ha,yf_agrc_pulses_tonne_ha,yf_agrc_rice_tonne_ha,yf_agrc_sugar_cane_tonne_ha,yf_agrc_tubers_tonne_ha,yf_agrc_vegetables_and_vines_tonne_ha,yf_lndu_supremum_pastures_tonne_per_ha
0,0,united_republic_of_tanzania,0,120147.615546,2.202670e+06,413139.184465,447873.760006,27593.507355,917029.317279,4.081904e+06,...,1.065225,2.155867,0.742475,5.173167,1.502500,1.6778,61.2569,5.41230,8.91720,92.81
1,0,united_republic_of_tanzania,1,121400.668717,2.225642e+06,417447.928860,452544.761029,27881.287780,926593.273157,4.124475e+06,...,1.035125,1.914367,0.767525,5.162867,1.522212,2.1449,65.5686,5.44554,8.87359,92.81
2,0,united_republic_of_tanzania,2,122160.077640,2.239564e+06,420059.229816,455375.606472,28055.696200,932389.478462,4.150275e+06,...,1.037275,1.997667,0.762400,5.291033,1.597687,2.2343,69.9105,4.60426,8.84238,92.81
3,0,united_republic_of_tanzania,3,124952.244504,2.290753e+06,429660.365351,465783.955121,28696.954676,953700.753440,4.245136e+06,...,1.044400,1.922367,0.780925,5.289600,1.575975,3.3060,69.9092,7.08912,8.85665,92.81
4,0,united_republic_of_tanzania,4,125211.561613,2.295507e+06,430552.052285,466750.610415,28756.510320,955679.996977,4.253946e+06,...,1.041000,1.896700,0.805300,5.295767,1.591163,3.3013,69.9100,6.94740,8.89455,92.81


In [5]:
df_primary =pd.read_csv(run.joinpath("ATTRIBUTE_PRIMARY.csv"))
df_primary.head()

,primary_id,design_id,strategy_id,future_id
0,0,0,0,0
1,69069,0,6002,0
2,70070,0,6003,0


In [6]:
df_strategy =pd.read_csv(run.joinpath("ATTRIBUTE_STRATEGY.csv"))
df_strategy.tail()

,strategy_id,strategy_code,strategy,description,transformation_specification,baseline_strategy_id
66,4006,IP:ALL,Sectoral Composite - IPPU,All (unique by transformer) IPPU transformations,TX:IPPU:DEC_CLINKER|TX:IPPU:DEC_DEMAND|TX:IPPU...,0
67,6000,PFLO:INC_HEALTHIER_DIETS,Singleton - Default Value - PFLO: Change diets,NaN,TX:PFLO:INC_HEALTHIER_DIETS,0
68,6001,PFLO:INC_IND_CCS,Singleton - Default Value - PFLO: Industrial c...,NaN,TX:PFLO:INC_IND_CCS,0
69,6002,PFLO:ALL,All Actions,All actions (unique by transformer),TX:AGRC:DEC_CH4_RICE|TX:AGRC:DEC_EXPORTS|TX:AG...,0
70,6003,PFLO:NDC,NDC,NDC,TX:TRNS:SHIFT_FUEL_MARITIME_STRATEGY_NDC|TX:AG...,0


In [7]:
# Merge the dataframes to include primary and strategy information in the ssp_output
ssp_output = ssp_output.merge(  df_primary, how='left', on='primary_id')
ssp_output = ssp_output.merge(  df_strategy, how='left', on='strategy_id')

In [8]:
ssp_output.value_counts('strategy')

strategy
All Actions         36
NDC                 36
Strategy TX:BASE    36
Name: count, dtype: int64

In [9]:
subsector_emission_fields = matt.get_all_subsector_emission_total_fields()
subsector_emission_fields

['emission_co2e_subsector_total_agrc',
 'emission_co2e_subsector_total_frst',
 'emission_co2e_subsector_total_lndu',
 'emission_co2e_subsector_total_lsmm',
 'emission_co2e_subsector_total_lvst',
 'emission_co2e_subsector_total_soil',
 'emission_co2e_subsector_total_waso',
 'emission_co2e_subsector_total_trww',
 'emission_co2e_subsector_total_ccsq',
 'emission_co2e_subsector_total_entc',
 'emission_co2e_subsector_total_fgtv',
 'emission_co2e_subsector_total_inen',
 'emission_co2e_subsector_total_scoe',
 'emission_co2e_subsector_total_trns',
 'emission_co2e_subsector_total_ippu']

In [10]:
ssp_output['year'] = ssp_output['time_period'] + 2015

In [11]:
# Melt ssp_output to long format for variables in subsector_emission_fields
ssp_long = ssp_output.melt(
    id_vars=['primary_id', 'strategy', 'strategy_id', 'year'],
    value_vars=subsector_emission_fields,
    var_name='Subsector',
    value_name='Emission'
)
ssp_long.head()

,primary_id,strategy,strategy_id,year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,emission_co2e_subsector_total_agrc,23.527001
1,0,Strategy TX:BASE,0,2016,emission_co2e_subsector_total_agrc,23.790862
2,0,Strategy TX:BASE,0,2017,emission_co2e_subsector_total_agrc,23.926033
3,0,Strategy TX:BASE,0,2018,emission_co2e_subsector_total_agrc,24.557659
4,0,Strategy TX:BASE,0,2019,emission_co2e_subsector_total_agrc,24.629395


In [12]:
ssp_long['Subsector'] = ssp_long['Subsector'].str.replace('emission_co2e_subsector_total_', '', regex=False)
ssp_long['Subsector'] = ssp_long['Subsector'].str.capitalize()
ssp_long.head()

,primary_id,strategy,strategy_id,year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,Agrc,23.527001
1,0,Strategy TX:BASE,0,2016,Agrc,23.790862
2,0,Strategy TX:BASE,0,2017,Agrc,23.926033
3,0,Strategy TX:BASE,0,2018,Agrc,24.557659
4,0,Strategy TX:BASE,0,2019,Agrc,24.629395


In [13]:
ssp_long.value_counts('Subsector')

Subsector
Agrc    108
Ccsq    108
Entc    108
Fgtv    108
Frst    108
Inen    108
Ippu    108
Lndu    108
Lsmm    108
Lvst    108
Scoe    108
Soil    108
Trns    108
Trww    108
Waso    108
Name: count, dtype: int64

In [14]:
subsector_dict = {
    "Agrc": "Agriculture",
    "Frst": "Forest",
    "Lndu": "Land Use",
    "Lsmm": "Livestock Manure Management",
    "Lvst": "Livestock",
    "Soil": "Soil Management",
    "Wail": "Liquid Waste",
    "Waso": "Solid Waste",
    "Trww": "Wastewater Treatment",
    "Ccsq": "Carbon Capture and Sequestration",
    "Enfu": "Energy Fuels",
    "Enst": "Energy Storage",
    "Entc": "Energy Technology",
    "Fgtv": "Fugitive Emissions",
    "Inen": "Industrial Energy",
    "Scoe": "Stationary Combustion and Other Energy",
    "Trns": "Transportation",
    "Trde": "Transportation Demand",
    "Ippu": "IPPU",
    "Econ": "Economy",
    "Gnrl": "General"
}

ssp_long['Subsector'] = ssp_long['Subsector'].replace(subsector_dict)
ssp_long.value_counts('Subsector')

Subsector
Agriculture                               108
Carbon Capture and Sequestration          108
Energy Technology                         108
Forest                                    108
Fugitive Emissions                        108
IPPU                                      108
Industrial Energy                         108
Land Use                                  108
Livestock                                 108
Livestock Manure Management               108
Soil Management                           108
Solid Waste                               108
Stationary Combustion and Other Energy    108
Transportation                            108
Wastewater Treatment                      108
Name: count, dtype: int64

In [15]:
ssp_long = ssp_long.rename(columns={"primary_id": "Primary Id",
                                    "strategy": "Strategy",
                                    "strategy_id": "Strategy Id",
                                    "year": "Year"})

In [16]:
ssp_long

,Primary Id,Strategy,Strategy Id,Year,Subsector,Emission
0,0,Strategy TX:BASE,0,2015,Agriculture,23.527001
1,0,Strategy TX:BASE,0,2016,Agriculture,23.790862
2,0,Strategy TX:BASE,0,2017,Agriculture,23.926033
3,0,Strategy TX:BASE,0,2018,Agriculture,24.557659
4,0,Strategy TX:BASE,0,2019,Agriculture,24.629395
...,...,...,...,...,...,...
1615,70070,NDC,6003,2046,IPPU,8.932291
1616,70070,NDC,6003,2047,IPPU,8.828117
1617,70070,NDC,6003,2048,IPPU,8.720757
1618,70070,NDC,6003,2049,IPPU,8.610186


In [17]:
ssp_long.to_csv(tableau.joinpath(f"emissions_{region}_250821.csv"), index=False)